In [ ]:
FLIGHT_PER_DAY = 6

AVERAGE_CUSTOMER_PER_FLIGHT = 120
AVERAGE_SEAT_PER_ACCOUNT = 4
AVERAGE_ACCOUNT_PER_FLIGHT = AVERAGE_CUSTOMER_PER_FLIGHT / AVERAGE_SEAT_PER_ACCOUNT

DAYS_PER_YEAR = 365

YEARS = 3

PERCENTAGE_OF_DUPLICATES_ACCOUNT = 0.5

ACCOUNTS_NEEDED = FLIGHT_PER_DAY * DAYS_PER_YEAR * YEARS * AVERAGE_ACCOUNT_PER_FLIGHT * (1 - PERCENTAGE_OF_DUPLICATES_ACCOUNT)
ACCOUNTS_NEEDED = int(ACCOUNTS_NEEDED)

In [ ]:
import json
from utils import unique
from typing import NamedTuple, Iterable


class Person(NamedTuple):
    username: str
    email: str
    password: str

NAMES_FILE = './member_data/names.json'
PASSWORD_FILE = "./member_data/passlist.json"

with open(NAMES_FILE, "r", encoding="utf-8") as nf, \
     open(PASSWORD_FILE, "r", encoding="utf-8") as pf:
    names = json.load(nf)
    passwords = json.load(pf)

@unique
def generate_random_name() -> tuple[str, str]:
    return tuple(random.sample(names, k=2))

@unique
def generate_random_phone_number() -> str:
    return f"({random.randint(1, 999):03}) {random.randint(1, 999):03}-{random.randint(1, 9999):04}"

def generate_random_gender() -> str:
    return "M" if random.randint(0, 1) else "F"

domains = ['gmail', 'yahoo', 'hotmail', 'outlook', 'yandex', 'protonmail', 'mail']
domains_weights = [1000, 200, 100, 200, 10, 20, 1]
tlds = ['com', 'net']
tld_weights = [20, 1]
@unique
def generate_random_email_address(first_name: str, last_name: str) -> str:
    domain = random.sample(domains, k=1, counts=domains_weights)[0]
    tld = random.sample(tlds, k=1, counts=tld_weights)[0]

    email = ""
    if random.randint(0, 1):
        email += f"{first_name.lower()}_"
    if not email or random.randint(0, 1):
        email += f"{last_name.lower().replace(".", "")}"
    email += f"@{domain}.{tld}"
    return email
    
def generate_person(n: int = 1000) -> Iterable[Person]:
    for _ in range(n):
        first_name, last_name = generate_random_name()

        email = generate_random_email_address(first_name, last_name)
        password = random.choice(passwords)

        yield Person((first_name + " " + last_name).title(), email, password)

In [ ]:
from typing import Iterable, NamedTuple
# import bcrypt
from datetime import timedelta, datetime
import random

class Account(NamedTuple):
    username: str
    hashed_password: str
    joined_date: datetime
    email_address: str
    phone_number: str
    gender: str
    
def get_random_time_offset(years: int = 3) -> timedelta:
    random_seconds = random.randint(0, years * 365 * 24 * 60 * 60)
    return timedelta(seconds=random_seconds)


def generate_random_hash() -> str:
    BASE_64 = "./ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
    # $2b$12$aIWCDww3E2lrCLY/rGWkrOxcgzKJ2R6cXFX/0jOpJ0Rdw.zZQNUPO
    return f"$2b$12${''.join(random.choices(BASE_64, k=53))}"


def generate_random_account(n: int) -> Iterable[Account]:
    now = datetime.now()
    for person in generate_person(n):
        yield Account(
            username=person.username,
            # hashed_password=bcrypt.hashpw(person["login"]["password"].encode("utf-8"), bcrypt.gensalt()).decode("utf-8"),
            hashed_password=generate_random_hash(),
            joined_date=now - get_random_time_offset(),
            email_address=person.email,
            phone_number=generate_random_phone_number(),
            gender=generate_random_gender()
        )

In [ ]:
from utils import paginate_insert_all
import re

trans = str.maketrans({"'": "''", "&": r"\&"})
make_sql_nice = lambda s: s.translate(trans)
sql_stmt = "    INTO ACCOUNT (USERNAME, HASHED_PASSWORD, JOINED_DATETIME, EMAIL_ADDRESS, PHONE_NUMBER, GENDER) " \
           "VALUES ('{}', '{}', TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), '{}', {}, {})"
email_pat = re.compile(r"^[a-z0-9!#$%&'*+/=?^_`{|}~-]+(\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*@(([a-z0-9]([a-z0-9-]*[a-z0-9])?\.)+[a-z0-9]([a-z0-9-]*[a-z0-9])?|\[((2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])\.){3}((2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:)\])$")
    
i = 0
with open("stuff.sql", "w") as f:
    def print_insert(account: Account):
        f.write(sql_stmt.format(make_sql_nice(account.username), make_sql_nice(account.hashed_password), account.joined_date, make_sql_nice(account.email_address), f"'{account.phone_number}'" if random.randint(0, 5) != 1 else 'NULL', f"'{account.gender}'" if random.randint(0, 10) != 1 else 'NULL') + '\n')
    print_insert = paginate_insert_all(f=print_insert, fp=f)
        
    for account in generate_random_account(ACCOUNTS_NEEDED):
        if not email_pat.match(account.email_address):
            print(account.email_address)
            continue
        
        print_insert(account)
        
    f.write("SELECT 1 FROM DUAL;")